# Formatting MODIS raw .hdf files

This script loads in the .hdf files with MODIS land cover type data downloaded from NASA EarthData repository on 24 October 2024. I decided to use Python because the method for processing the data in Python is much more straightforward than for R. One the data is in array format, I save it and transfer these formatted data to R for the rest of the analysis using the pickle package.

First, I am importing the packages that are necessary for this pipeline, notable the xarray package is what processes the .hdf file format.

In [1]:
# Import packages
import os
import warnings
from glob import glob
import pickle

import numpy as np
import xarray as xr
import rioxarray as rxr

Next, I am defining the datasets that are necessary to extract from each .hdf file. These are as follows:

* LC_Type1: annual IGBP classification
* LC_Type2: annual UMD classification
* LC_Type3: annual LAI classification
* QC: Product quality flags
* LW: Binary land (class 2)/water (class 1) mask derived from MOD44W

The follow datasets are dropped:

* LC_Type4: annual BGC classification, because it does not include savanna or grassland
* LC_Type5: annual PFT classification, because it does not incldue savanna or grassland
* LC_Prop1: LCCS1 land cover layer, because it does not include savanna or cropland (so converting plant types to ecosystems would be impossible if we don't know what corresponds to cropland)
* LC_Prop2: LCCS2 land use layer, because it does not include savanna or prairie
* LC_Prop3: LCCS3 surface hydrology layer, because it does not include savanna
* LC_Prop1_Ass: LCCS1 land cover layer confidence, because I'm not using LC_Prop1
* LC_Prop2_Ass: LCCS2 land use layer confidence, because I'm not using LC_Prop2
* LC_Prop3_Ass: LCCS3 surface hydrology layer confidence, because I'm not using LC_Prop3

In [2]:
# Names of datasets we want to keep from the data
desired_datasets = ['LC_Type1',
                    'LC_Type2',
                    'LC_Type3',
                    'QC',
                    'LW']

Now, I'm defining a function that we can loop over for formatting the data, including loading in the desired datasets, reprojecting the data, and converting it to an array format.

In [3]:
# Function for formatting data
def data_func(i):
    src = rxr.open_rasterio(file_list[i],
                            masked = True,
                            variable = desired_datasets).squeeze()
    src_proj = src.rio.reproject('EPSG:3175')
    src_array = xr.Dataset.to_array(src_proj)
    return(src_array)

## Cell 1: h10v04

Because each cell has different coordinates, we need to process the data for each cell separately. Therefore, I previously divided all the downloaded datasets into five cell subdirectories manually. That is, I used the download script included in the MODIS/ directory to dump all files in the MODIS directory, then I moved the files into different subdirectories based on their horizontal and vertical tile numbers.

Here, we start with cell h10v04.

In [4]:
# Defining directory and data files for first cell
data_dir = '/Volumes/FileBackup/SDM_bigdata/MODIS/h10v04'
# use pathlib.path
file_list = glob(os.path.join(data_dir, '*.hdf'))

# Vector of file index for this cell
ii = np.arange(np.shape(file_list)[0])
ii

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22])

Now, I use the function I defined above to loop over all the individual .hdf files for each year's data product in the same cell.

In [5]:
# Loop over all files in this cell and put into list format
list_h10v04 = [data_func(i) for i in ii]

Since all the files have the same format, we can convert it to an array, which allows us to manipulate it in R.

In [6]:
# Convert to numpy array
array_h10v04 = np.array(list_h10v04)

Here, I'm checking the dimensions to make sure it makes sense.

In [7]:
# Check shape
# 23 = years
# 5 = datasets
# 2201 x 2584 = coordinates
np.shape(array_h10v04)

(23, 5, 2201, 2584)

Now, I'm extracting the coordinates for two samples of the files. I make sure that they're identical before moving on.

In [8]:
# Extract latitude and longitude
samp1 = rxr.open_rasterio(file_list[0], masked = True, variable = desired_datasets).squeeze()
samp2 = rxr.open_rasterio(file_list[10], masked = True, variable = desired_datasets).squeeze()

samp1_reprojected = samp1.rio.reproject('EPSG:3175')
samp2_reprojected = samp2.rio.reproject('EPSG:3175')

lat_1 = samp1_reprojected['y']
lat_2 = samp2_reprojected['y']
lon_1 = samp1_reprojected['x']
lon_2 = samp2_reprojected['x']

In [9]:
# Check to make sure that the lat & lons are the same 
print(np.array_equal(lat_1, lat_2)) # should be True
print(np.array_equal(lon_1, lon_2)) # should be True

print(len(lat_1))
print(len(lon_1))

True
True
2201
2584


Finally, I format and save the coordinates and datasets for this cell.

In [10]:
# Define objects to save
lats = np.array(lat_1)
lons = np.array(lon_1)
modis_h10v04 = array_h10v04

In [11]:
# Save in pickle files
filehandler = open('../../data/intermediate/MODIS/lats_h10v04.pickle', 'wb')
pickle.dump(lats, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/lons_h10v04.pickle', 'wb')
pickle.dump(lons, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/modis_h10v04.pickle', 'wb')
pickle.dump(modis_h10v04, filehandler)
filehandler.close()

## Cell 2: h10v05

Now that I've run through things once, I'm not going to explain the invidiual steps. I'll just repeat them.

In [12]:
data_dir = '/Volumes/FileBackup/SDM_bigdata/MODIS/h10v05'
file_list = glob(os.path.join(data_dir, '*.hdf'))

ii = np.arange(np.shape(file_list)[0])
ii

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22])

In [13]:
list_h10v05 = [data_func(i) for i in ii]

In [14]:
array_h10v05 = np.array(list_h10v05)

In [15]:
np.shape(array_h10v05)

(23, 5, 1862, 2841)

In [16]:
samp1 = rxr.open_rasterio(file_list[0], masked = True, variable = desired_datasets).squeeze()
samp2 = rxr.open_rasterio(file_list[10], masked = True, variable = desired_datasets).squeeze()

samp1_reprojected = samp1.rio.reproject('EPSG:3175')
samp2_reprojected = samp2.rio.reproject('EPSG:3175')

lat_1 = samp1_reprojected['y']
lat_2 = samp2_reprojected['y']
lon_1 = samp1_reprojected['x']
lon_2 = samp2_reprojected['x']

In [17]:
print(np.array_equal(lat_1, lat_2))
print(np.array_equal(lon_1, lon_2))

print(len(lat_1))
print(len(lon_1))

True
True
1862
2841


In [18]:
lats = np.array(lat_1)
lons = np.array(lon_1)
modis_h10v05 = array_h10v05

In [19]:
filehandler = open('../../data/intermediate/MODIS/lats_h10v05.pickle', 'wb')
pickle.dump(lats, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/lons_h10v05.pickle', 'wb')
pickle.dump(lons, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/modis_h10v05.pickle', 'wb')
pickle.dump(modis_h10v05, filehandler)
filehandler.close()

## Cell 3: h11v04

In [21]:
data_dir = '/Volumes/FileBackup/SDM_bigdata/MODIS/h11v04'
file_list = glob(os.path.join(data_dir, '*.hdf'))

ii = np.arange(np.shape(file_list)[0])
ii

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22])

In [23]:
list_h11v04 = [data_func(i) for i in ii]

In [24]:
array_h11v04 = np.array(list_h11v04)

In [25]:
np.shape(array_h11v04)

(23, 5, 1811, 2881)

In [27]:
samp1 = rxr.open_rasterio(file_list[0], masked = True, variable = desired_datasets).squeeze()
samp2 = rxr.open_rasterio(file_list[10], masked = True, variable = desired_datasets).squeeze()

samp1_reprojected = samp1.rio.reproject('EPSG:3175')
samp2_reprojected = samp2.rio.reproject('EPSG:3175')

lat_1 = samp1_reprojected['y']
lat_2 = samp2_reprojected['y']
lon_1 = samp1_reprojected['x']
lon_2 = samp2_reprojected['x']

In [28]:
print(np.array_equal(lat_1, lat_2))
print(np.array_equal(lon_1, lon_2))

print(len(lat_1))
print(len(lon_1))

True
True
1811
2881


In [30]:
lats = np.array(lat_1)
lons = np.array(lon_1)
modis_h11v04 = array_h11v04

In [31]:
filehandler = open('../../data/intermediate/MODIS/lats_h11v04.pickle', 'wb')
pickle.dump(lats, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/lons_h11v04.pickle', 'wb')
pickle.dump(lons, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/modis_h11v04.pickle', 'wb')
pickle.dump(modis_h11v04, filehandler)
filehandler.close()

## Cell 4: h11v05

In [32]:
data_dir = '/Volumes/FileBackup/SDM_bigdata/MODIS/h11v05'
file_list = glob(os.path.join(data_dir, '*.hdf'))

ii = np.arange(np.shape(file_list)[0])
ii

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22])

In [33]:
list_h11v05 = [data_func(i) for i in ii]

In [34]:
array_h11v05 = np.array(list_h11v05)

In [35]:
np.shape(array_h11v05)

(23, 5, 1657, 3052)

In [36]:
samp1 = rxr.open_rasterio(file_list[0], masked = True, variable = desired_datasets).squeeze()
samp2 = rxr.open_rasterio(file_list[10], masked = True, variable = desired_datasets).squeeze()

samp1_reprojected = samp1.rio.reproject('EPSG:3175')
samp2_reprojected = samp2.rio.reproject('EPSG:3175')

lat_1 = samp1_reprojected['y']
lat_2 = samp2_reprojected['y']
lon_1 = samp1_reprojected['x']
lon_2 = samp2_reprojected['x']

In [38]:
print(np.array_equal(lat_1, lat_2))
print(np.array_equal(lon_1, lon_2))

print(len(lat_1))
print(len(lon_1))

True
True
1657
3052


In [39]:
lats = np.array(lat_1)
lons = np.array(lon_1)
modis_h11v05 = array_h11v05

In [40]:
filehandler = open('../../data/intermediate/MODIS/lats_h11v05.pickle', 'wb')
pickle.dump(lats, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/lons_h11v05.pickle', 'wb')
pickle.dump(lons, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/modis_h11v05.pickle', 'wb')
pickle.dump(modis_h11v05, filehandler)
filehandler.close()

## Cell 5: h12v04

In [41]:
data_dir = '/Volumes/FileBackup/SDM_bigdata/MODIS/h12v04'
file_list = glob(os.path.join(data_dir, '*.hdf'))

ii = np.arange(np.shape(file_list)[0])
ii

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22])

In [42]:
list_h12v04 = [data_func(i) for i in ii]

In [43]:
array_h12v04 = np.array(list_h12v04)

In [44]:
np.shape(array_h12v04)

(23, 5, 1582, 3109)

In [45]:
samp1 = rxr.open_rasterio(file_list[0], masked = True, variable = desired_datasets).squeeze()
samp2 = rxr.open_rasterio(file_list[10], masked = True, variable = desired_datasets).squeeze()

samp1_reprojected = samp1.rio.reproject('EPSG:3175')
samp2_reprojected = samp2.rio.reproject('EPSG:3175')

lat_1 = samp1_reprojected['y']
lat_2 = samp2_reprojected['y']
lon_1 = samp1_reprojected['x']
lon_2 = samp2_reprojected['x']

In [46]:
print(np.array_equal(lat_1, lat_2))
print(np.array_equal(lon_1, lon_2))

print(len(lat_1))
print(len(lon_1))

True
True
1582
3109


In [47]:
lats = np.array(lat_1)
lons = np.array(lon_1)
modis_h12v04 = array_h12v04

In [49]:
filehandler = open('../../data/intermediate/MODIS/lats_h12v04.pickle', 'wb')
pickle.dump(lats, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/lons_h12v04.pickle', 'wb')
pickle.dump(lats, filehandler)
filehandler.close()

filehandler = open('../../data/intermediate/MODIS/modis_h12v04.pickle', 'wb')
pickle.dump(modis_h12v04, filehandler)
filehandler.close()